In [1]:
import pandas as pd
from tqdm.auto import tqdm
import json

In [4]:
# 先构建 Train / Test
df_foldseek = pd.read_csv("clu_cluster.tsv", sep='\t', names=["rep", "member"])
df_foldseek["rep"] = df_foldseek["rep"].apply(lambda x: x.lower())
df_foldseek["member"] = df_foldseek["member"].apply(lambda x: x.lower())
df_foldseek["rep_id"] = df_foldseek["rep"].apply(lambda x: x.split("_")[0].lower())
df_foldseek["member_id"] = df_foldseek["member"].apply(lambda x: x.split("_")[0].lower())
print(len(df_foldseek))
df_foldseek.head()

56397


,rep,member,rep_id,member_id
0,1a06,1a06,1a06,1a06
1,1a09,1a09,1a09,1a09
2,1a09,1a07,1a09,1a07
3,1a09,1a08,1a09,1a08
4,1a09,1a1a,1a09,1a1a


In [ ]:
df_cluster = pd.read_csv("clusterRes_cluster.tsv", sep='\t', names=["rep", "member"])
df_cluster["rep"] = df_cluster["rep"].apply(lambda x: x.lower())
df_cluster["member"] = df_cluster["member"].apply(lambda x: x.lower())
df_cluster["rep_id"] = df_cluster["rep"].apply(lambda x: x.split("_")[0].lower())
df_cluster["member_id"] = df_cluster["member"].apply(lambda x: x.split("_")[0].lower())
df_cluster.head()

df_query = pd.read_csv("../job_list.csv")
df_query_set = set(df_query["pdb_id"])
print(df_query_set)

# 初始化集合
test_set = set()
train_set = set()

for pdb_id in tqdm(df_query_set):
    filter_df = df_cluster[df_cluster["member_id"] == pdb_id]
    cluster_ids = filter_df["rep_id"].unique()
    if all(rep_id in df_query_set for rep_id in cluster_ids):
        test_set.add(pdb_id)
    else:
        train_set.add(pdb_id)

print(f"test set: {len(test_set)}, train set: {len(train_set)}")

print(test_set)

redundant_set = set()
non_redundant_set = set()

for pdb_id in tqdm(df_query_set):
    filter_df = df_cluster[df_cluster["member_id"] == pdb_id]
    cluster_ids = filter_df["rep_id"].unique()

    if len(cluster_ids) == 1:
        if cluster_ids[0] == pdb_id:
            non_redundant_set.add(pdb_id)
        elif cluster_ids[0] in df_query_set:
            redundant_set.add(pdb_id)
        else:
            non_redundant_set.add(pdb_id)
    else:
        if any(rep_id in df_query_set for rep_id in cluster_ids):
            redundant_set.add(pdb_id)
        else:
            non_redundant_set.add(pdb_id)
print(
    f"redundant set: {len(redundant_set)}, non-redundant set: {len(non_redundant_set)}"
)
print(non_redundant_set)
core_set = non_redundant_set & test_set
print(f"test independent set: {len(core_set)}")

  0%|          | 0/259 [00:00<?, ?it/s]

redundant set: 38, non-redundant set: 221


In [36]:
with open("clusters_query.json", "w") as f:
    json.dump(
        {
            "test_set": list(test_set),
            "train_set": list(train_set),
            "redundant_set": list(redundant_set),
            "non_redundant_set": list(non_redundant_set),
            "core_set": list(core_set),
        }, f, indent=4
    )